<a href="https://colab.research.google.com/github/vvivvi/kaggle-c1/blob/master/C1_hyperpaparameter_tuning_level1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install catboost

     |████████████████████████████████| 64.8MB 47kB/s 


In [0]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 2.2MB/s 


Now that we have fixed and generated three feature subsets
1. non-lagged + lagged textual features
2. lagged {target,item,shop} + non-lagged basic categories
3. lagged features within shop

and three first level classifiers types for each
* a.  CatBoost
* b. RidgeCV 
* c. Random Forest (sklearn) 

we search for hyperarameters that are used for predicting a month 
based on twelve month history, with one month gap between training and prediction periods.

This is a compromise of the prediction quality on the other hand, and not having the prediction 
quality and optimal hyperparameters vary too much over the training period when generating the first level predictions as input features of second stacking level.

The search for hyperparameters is problematic in whole because the chosen validation scheme is lacking. There may not be
too much that can be done, because the validation data necessarily has different distribution as the actual testing data.
This is because the temporal nature of the prediction problem. The distributions slowly drift during cause of time. Therefore, 
it is good to have the validation period temporally close to the test period. On the other hand, data analysis shows strong seasonal=(yearly) effects. 
Predicting October sales based on previous year simply is a very different problem to predicting December sales, as sales figures seem to peak strongly in December and have special characteristics.

We decide to search for such hyperparameters that maximise the quality of predictions (with
reasonable computational burden) in the hold-out validation data of Oct 2015. This is despite the fact that we have seen in examples that
such optimal model hyperparameters do not result in optimal prediction quality for Dec 2015.
We specifically do not search for such hyperparameters (via a coross-validation scheme) that would maximise the quality of predictions during
the training period, as the value of temporally distant predictions is questionable after because of the distribution shift throughtime.


The parameters are used for
a) creating submissions for ensembling using simple schemes
b) generating level 2 input features for a stacking algorithm



In [0]:
import sys
import os.path
import json
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
from itertools import product
import gc
from tqdm import tqdm_notebook
import re
from catboost import CatBoostRegressor, Pool

from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer

from time import time
import pprint


IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive') 
  if not os.path.isfile('SETTINGS.json'):
       # hard coded data directory in drive is used if SETTINGS.json not present 
       config={}
       config['DATA_DIR'] = '/content/gdrive/My Drive/kaggle-c1'
       with open('SETTINGS.json', 'w') as outfile:
         json.dump(config, outfile)

with open('SETTINGS.json') as config_file:
    config = json.load(config_file)

DATA_DIR = config['DATA_DIR']

print('Using DATA_DIR ', DATA_DIR)

DATA_FOLDER = DATA_DIR

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Using DATA_DIR  /content/gdrive/My Drive/kaggle-c1


In [0]:
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

index_cols=['item_id','shop_id','date_block_num']
date_block_val = 33
date_block_test = 35 # Dec 2015

In [0]:
# a wrapper class to use pre-defined division to training and hold-out set
# as a cross-validation object

class HoldOut:
    """
    Hold-out cross-validator generator. In the hold-out, the
    data is split only once into a train set and a test set.
    Here the split is given as a input parameter in the class initialisation
    Unlike in other cross-validation schemes, the hold-out
    consists of only one iteration.

    Parameters
    ----------
    train_indices, test_indices : the class just passes on these when yielding splits


    """

    def __init__(self, train_indices, test_indices):
        self.train_indices = train_indices
        self.test_indices = test_indices

    def __iter__(self):
        yield self.train_indices, self.test_indices

Define helper functions

In [0]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [0]:
def write_predictions_by_array(array, filename):
  df=pd.DataFrame(array)
  df.columns=['item_cnt_month']
  df.to_csv(os.path.join(DATA_FOLDER, filename), index_label='ID')

In [0]:
def clipped_rmse(gt, predicted,clip_min=0, clip_max=20):
  target=np.minimum(np.maximum(gt,clip_min), clip_max)
  return np.sqrt((target-predicted)**2).mean()

In [0]:
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params
    

In [0]:
def find_catboost_parameters_bayes(feature_file_name, search_space=None, n_iter=100):
    all_data = pd.read_csv(os.path.join(DATA_FOLDER, feature_file_name))

    dates=all_data['date_block_num']

    dates_train = (dates>= date_block_val - 13) & (dates<= date_block_val - 2)
    dates_trainval = (dates>= date_block_test - 13) & (dates<= date_block_test - 2)

# extract training, validation and test sets (labels and features)

    y_train=all_data.loc[dates_train, 'target']
    y_trainval=all_data.loc[dates_trainval, 'target']
    y_val = all_data.loc[dates == date_block_val, 'target']

    to_drop_cols = ['target','date_block_num']

    X_train = all_data.loc[dates_train].drop(to_drop_cols, axis=1)
    X_trainval = all_data.loc[dates_trainval].drop(to_drop_cols, axis=1)
    X_val = all_data.loc[dates == date_block_val].drop(to_drop_cols, axis=1)
    X_test = all_data.loc[dates == date_block_test].drop(to_drop_cols, axis=1)

    # determine how to permute test set predictions for submission generation 

    shop_item2submissionid={}
    for idx, row in test_spec.iterrows():
        shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])] = row['ID']

    test_data=all_data.loc[dates == date_block_test, ['shop_id','item_id']]    

    testidx2submissionidx=np.zeros(test_data.shape[0], dtype=np.int32)
    for idx in range(test_data.shape[0]):
        row =test_data.iloc[idx]
        testidx2submissionidx[idx] = shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])]

    #invert the mapping
    submissionidx2testidx=np.zeros(test_data.shape[0], dtype=np.int32)
    for i in range(test_data.shape[0]):
        submissionidx2testidx[testidx2submissionidx[i]]=i

    del test_data
    gc.collect()    

    X_paramsearch =  pd.concat([X_train, X_val],ignore_index=True)
    y_paramsearch = pd.concat([y_train, y_val],ignore_index=True)
    
    train_indices = np.arange(X_train.shape[0])
    val_indices = np.arange(X_val.shape[0]) + X_train.shape[0]
    
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    if search_space is None: 
        search_space = {'iterations': Integer(300, 1500),
                        'depth': Integer(8, 16),
                        'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                        'random_strength': Real(1, 100, 'log-uniform'),
                        'l2_leaf_reg': Real(0.001, 2.0, 'log-uniform'),
                        }

    clf = CatBoostRegressor(task_type='GPU',has_time=True, verbose=False)


    # Setting up BayesSearchCV

    cv = HoldOut(train_indices=train_indices, test_indices=val_indices)

    opt = BayesSearchCV(clf,
                        search_space,
                        scoring=mse_scorer,
                        cv=cv,
                        n_iter=n_iter,
                        n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
                        return_train_score=False,
                        refit=False,
                        optimizer_kwargs={'base_estimator': 'GP'})

    best_params = report_perf(opt, X_paramsearch, y_paramsearch,'CatBoost', 
                               callbacks=[VerboseCallback(100)])
    
    return best_params

# Feature set 1: non-lagged and lagged basic categories

In [0]:
#best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv')
#print(best_params_basic)
# CatBoost took 5376.07 seconds,  candidates checked: 100, best CV score: -20.311 ± 0.000
#Best parameters:
#OrderedDict([('depth', 9),
#             ('iterations', 339),
#             ('l2_leaf_reg', 0.1043327165183886),
#             ('learning_rate', 0.03646613151300171),
#             ('random_strength', 70.08445579850765)])
#
# OrderedDict([('depth', 9), ('iterations', 339), ('l2_leaf_reg', 0.1043327165183886), ('learning_rate', 0.03646613151300171), ('random_strength', 70.08445579850765)])

#CatBoost took 6637.58 seconds,  candidates checked: 100, best CV score: -19.772 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 300),
#             ('l2_leaf_reg', 2.0),
#             ('learning_rate', 0.08922738789933286),
#             ('random_strength', 100.0)])
#
#OrderedDict([('depth', 8), ('iterations', 300), ('l2_leaf_reg', 2.0), ('learning_rate', 0.08922738789933286), ('random_strength', 100.0)])

#CatBoost took 4328.61 seconds,  candidates checked: 100, best CV score: -17.769 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 1500),
#             ('l2_leaf_reg', 0.001),
#             ('learning_rate', 0.1813605946638354),
#             ('random_strength', 100.0)])
#
#OrderedDict([('depth', 8), ('iterations', 1500), ('l2_leaf_reg', 0.001), ('learning_rate', 0.1813605946638354), ('random_strength', 100.0)])
#CatBoost took 5568.42 seconds,  candidates checked: 100, best CV score: -17.705 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 300),
#             ('l2_leaf_reg', 0.001),
#             ('learning_rate', 0.21121409098653413),
#             ('random_strength', 100.0)])
#
#OrderedDict([('depth', 8), ('iterations', 300), ('l2_leaf_reg', 0.001), ('learning_rate', 0.21121409098653413), ('random_strength', 100.0)])

In [0]:
# from the best point found, start line searches wrt. one parameter at time
# start from ones that reach the limit (=random strength, l2_leaf_reg)
#search_space = {'iterations': Categorical([1500]),   
#                'depth': Categorical([8]), 
#                'learning_rate': Categorical([0.1813605946638354]),
#                'random_strength': Real(80,3000,'log-uniform'),
#                'l2_leaf_reg': Categorical([0.001]),
#                }

#best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv', search_space=search_space, n_iter=20)
#print(best_params_basic)

#CatBoost took 1424.11 seconds,  candidates checked: 20, best CV score: -18.071 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 1500),
#             ('l2_leaf_reg', 0.001),
#             ('learning_rate', 0.1813605946638354),
#             ('random_strength', 80.0)])
#
#OrderedDict([('depth', 8), ('iterations', 1500), ('l2_leaf_reg', 0.001), ('learning_rate', 0.1813605946638354), ('random_strength', 80.0)])



SyntaxError: ignored

In [0]:
search_space = {'iterations': Categorical([1500]),   
                'depth': Categorical([8]), 
                'learning_rate': Categorical([0.1813605946638354]),
                'random_strength': Real(1,80,'log-uniform'),
                'l2_leaf_reg': Categorical([0.001]),
                }

best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv', search_space=search_space, n_iter=20)
print(best_params_basic)

#CatBoost took 1431.23 seconds,  candidates checked: 20, best CV score: -18.482 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 1500),
#             ('l2_leaf_reg', 0.001),
#             ('learning_rate', 0.1813605946638354),
#             ('random_strength', 79.99999999999999)])

#OrderedDict([('depth', 8), ('iterations', 1500), ('l2_leaf_reg', 0.001), ('learning_rate', 0.1813605946638354), ('random_strength', 79.99999999999999)])


In [0]:
#search_space = {'iterations': Categorical([1500]),   
#                'depth': Categorical([8]), 
#                'learning_rate': Categorical([0.1813605946638354]),
#                'random_strength': Categorical([80]),
#                'l2_leaf_reg': Real(0.00001,0.1,'log-uniform'),
#                }#
#
#best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv', search_space=search_space, n_iter=20)
#print(best_params_basic)

#CatBoost took 1426.77 seconds,  candidates checked: 20, best CV score: -17.929 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 1500),
#             ('l2_leaf_reg', 0.005744167167485476),
#             ('learning_rate', 0.1813605946638354),
#             ('random_strength', 80)])
#
#OrderedDict([('depth', 8), ('iterations', 1500), ('l2_leaf_reg', 0.005744167167485476), ('learning_rate', 0.1813605946638354), ('random_strength', 80)])

In [0]:
#search_space = {'iterations': Integer(100,2000,'log-uniform'),   
#                'depth': Categorical([8]), 
#                'learning_rate': Categorical([0.1813605946638354]),
#                'random_strength': Categorical([80]),
#                'l2_leaf_reg': Categorical([ 0.005744167167485476]),
#                }
#
#best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv', search_space=search_space, n_iter=20)
#print(best_params_basic)

#CatBoost took 901.52 seconds,  candidates checked: 20, best CV score: -17.552 ± 0.000
#Best parameters:
#rderedDict([('depth', 8),
#             ('iterations', 107),
#             ('l2_leaf_reg', 0.005744167167485476),
#             ('learning_rate', 0.1813605946638354),
#             ('random_strength', 80)])

# OrderedDict([('depth', 8), ('iterations', 107), ('l2_leaf_reg', 0.005744167167485476), ('learning_rate', 0.1813605946638354), ('random_strength', 80)])

In [0]:
#search_space = {'iterations': Categorical([107]),   
#                'depth': Categorical([8]), 
#                'learning_rate': Real(0.1,0.3,'uniform'),
#                'random_strength': Categorical([80]),
#                'l2_leaf_reg': Categorical([ 0.005744167167485476]),
#                }
#
#best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv', search_space=search_space, n_iter=20)
#print(best_params_basic)
#
#Iteration No: 21 started. Searching for the next optimal point.
#CatBoost took 146.82 seconds,  candidates checked: 20, best CV score: -17.566 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 107),
#             ('l2_leaf_reg', 0.005744167167485476),
#             ('learning_rate', 0.23068135624313127),
#             ('random_strength', 80)])
#
#OrderedDict([('depth', 8), ('iterations', 107), ('l2_leaf_reg', 0.005744167167485476), ('learning_rate', 0.23068135624313127), ('random_strength', 80)])


Experiment with some additional parameters

In [0]:
#search_space = {'iterations': Categorical([107]),   
#                'depth': Categorical([8]), 
#                'learning_rate': Categorical([0.181]),
#                'random_strength': Categorical([80]),
#                'l2_leaf_reg': Categorical([ 0.005744167167485476]),
#                'bagging_temperature': Real(0.01,100,'log-uniform')
#                }
#best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv', search_space=search_space, n_iter=20)
#print(best_params_basic)

#CatBoost took 141.68 seconds,  candidates checked: 20, best CV score: -17.738 ± 0.000
#Best parameters:
#OrderedDict([('bagging_temperature', 0.9784460719042715),
#             ('depth', 8),
#             ('iterations', 107),
#             ('l2_leaf_reg', 0.005744167167485476),
#             ('learning_rate', 0.181),
#            ('random_strength', 80)])
# OrderedDict([('bagging_temperature', 0.9784460719042715), ('depth', 8), ('iterations', 107), ('l2_leaf_reg', 0.005744167167485476), ('learning_rate', 0.181), ('random_strength', 80)])

# no improvement from here


In [0]:
search_space = {'iterations': Categorical([107]),   
                'depth': Categorical([8]), 
                'learning_rate': Categorical([0.181]),
                'random_strength': Categorical([80]),
                'l2_leaf_reg': Categorical([ 0.005744167167485476]),
                'grow_policy': Categorical(['SymmetricTree','Depthwise','Lossguide'])
                }
best_params_basic = find_catboost_parameters_bayes('feature_set_basic.csv', search_space=search_space, n_iter=20)
print(best_params_basic)


In [0]:
best_params_text = find_catboost_parameters_bayes('feature_set_text.csv')
print(best_params_text)

#Best parameters:
#OrderedDict([('depth', 16),
#             ('iterations', 300),
#             ('l2_leaf_reg', 0.001),
#             ('learning_rate', 0.17754758868410853),
#             ('random_strength', 1.0)])
#
#OrderedDict([('depth', 16), ('iterations', 300), ('l2_leaf_reg', 0.001), ('learning_rate', 0.17754758868410853), ('random_strength', 1.0)])


In [0]:
best_params_within = find_catboost_parameters_bayes('feature_set_within.csv')
print(best_params_within)
#CatBoost took 4357.06 seconds,  candidates checked: 100, best CV score: -21.848 ± 0.000
#Best parameters:
#OrderedDict([('depth', 8),
#             ('iterations', 1500),
#             ('l2_leaf_reg', 2.0),
#             ('learning_rate', 0.058681717789493895),
#             ('random_strength', 2.168205069150344)])
#
# OrderedDict([('depth', 8), ('iterations', 1500), ('l2_leaf_reg', 2.0), ('learning_rate', 0.058681717789493895), ('random_strength', 2.168205069150344)])

In [0]:
to_drop_cols=[] # [col for col in X_train.columns.values if re.search('internet',col)]
reg=CatBoostRegressor(task_type='GPU', iterations=100, eta=0.3,depth=10, metric_period=20)
reg.fit(X_trainval.drop(to_drop_cols,axis=1).to_numpy(), y_trainval)
pred_test = np.clip(reg.predict(X_test.drop(to_drop_cols,axis=1).to_numpy()), 0, 20)
write_predictions_by_array(pred_test[submissionidx2testidx], 'submission-catboost-feature_set_basic.csv')

# LB  1.008784 and 1.027125 (d=8, iterations=100, eta=0.3)
# 1.046352 and 1.054449 (d=10, iterations=100, eta=0.3)